In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import scipy.stats as stats

In [3]:
from numpy.random import RandomState

## Additional mortality calculation:
$$D_u = s_f D_c$$

where $D_c$ is the current number of annual deaths in the impacted population $P$ and $D_u$ is the number of annual deaths expected after the policy change. 

The scaling factor $s_f$ is calculated based on the change in proportion of the impacted population who are insured (proportion of currently insured $p^c_i$ to proportion inusred $p^u_i$ after policy change) and is given by

$$s_f = \frac{p^u_i + \lambda (1-p^u_i)}{p^c_i + \lambda (1-p^c_i)}$$

where $\lambda$ is hazard ratio of death for those uninsured compared with those insured.

Additional mortality is then givne by $$D_u - D_c$$.

## Work requirement
+ Impacted population $P$ are individuals in age-group 19-64. We use data from [National Population Projection Datasets](https://www.census.gov/data/datasets/2023/demo/popproj/2023-popproj.html) to inform it. **Updated data**.
+ We caculate $D_c$ based on death rate $d_r$ for the impacted population $P$ using [NCHS Data Brief 2024](https://www.cdc.gov/nchs/products/databriefs/db492.htm) which provides latest estimates(2022). So, $$D_c = d_r P $$. **Updated data**.
+ We inform the proportion of individuals with insurance $p^c_i$ in the impacted population $P$ using [2023 American Community Survey Data](https://data.census.gov/table/ACSST5Y2023.S2701?q=health%20insurance) from United States Census Bureau. **Updated data**.
+ To inofrm, $p^u_i$ the updated proportion of insured impacted population we use [this report](https://www.cbpp.org/research/health/medicaid-work-requirements-could-put-36-million-people-at-risk-of-losing-health) from Center for Budget and Policies Priorities. **New data**.
+ Hazard ratio ($\lambda$) is informed from [Wilper et. al. 2003, Am J Public Health](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2775760/).


## Questions?
+ Do I have enough data to do it at state-level.
+ How does our past analysis relate to this
 


## Data

In [4]:
# read population data (https://www.census.gov/data/datasets/2023/demo/popproj/2023-popproj.html)
pop_ = pd.read_csv('data/np2023.csv') # population projection data  from US Census Bureau
# NOTE 1: this doesn't have state-level data
# Note 2: there is an update estimate for projection data.

In [5]:
# read insurance data from US Census Bureau (https://data.census.gov/table/ACSST5Y2023.S2701?q=health%20insurance) 
# 2023: ACS 5-year estimates subject table
ins_ = pd.read_excel('data/ACS ST 5Y 2023 Data.xlsx',sheet_name='Data',header=[0,1,2],index_col=0)
ins_

United States  \
                                                           Total   
Label                                                   Estimate   
Civilian noninstitutionalized population             327,425,278   
AGE                                                          NaN   
Under 6 years                                         22,860,400   
6 to 18 years                                         55,187,398   
19 to 25 years                                        29,463,133   
...                                                          ...   
Civilian noninstitutionalized population for wh...   323,581,141   
Below 138 percent of the poverty threshold            59,207,745   
138 to 399 percent of the poverty threshold          128,737,340   
At or above 400 percent of the poverty threshold     135,636,056   
Below 100 percent of the poverty threshold            40,364,323   

                                                                    \
                                                                     
Label                                              Margin of Error   
Civilian noninstitutionalized population                    ±8,784   
AGE                                                            NaN   
Under 6 years                                              ±16,848   
6 to 18 years                                              ±22,833   
19 to 25 years                                             ±25,847   
...                                                            ...   
Civilian noninstitutionalized population for wh...         ±18,635   
Below 138 percent of the poverty threshold                ±374,885   
138 to 399 percent of the poverty threshold               ±210,228   
At or above 400 percent of the poverty threshold          ±548,028   
Below 100 percent of the poverty threshold                ±273,055   

                                                                 \
                                                        Insured   
Label                                                  Estimate   
Civilian noninstitutionalized population            299,424,402   
AGE                                                         NaN   
Under 6 years                                        21,837,881   
6 to 18 years                                        52,000,934   
19 to 25 years                                       25,345,552   
...                                                         ...   
Civilian noninstitutionalized population for wh...  295,817,469   
Below 138 percent of the poverty threshold           50,532,252   
138 to 399 percent of the poverty threshold         114,783,228   
At or above 400 percent of the poverty threshold    130,501,989   
Below 100 percent of the poverty threshold           34,365,013   

                                                                    
                                                                    
Label                                              Margin of Error  
Civilian noninstitutionalized population                  ±245,709  
AGE                                                            NaN  
Under 6 years                                              ±21,655  
6 to 18 years                                              ±33,152  
19 to 25 years                                             ±29,354  
...                                                            ...  
Civilian noninstitutionalized population for wh...        ±253,096  
Below 138 percent of the poverty threshold                ±289,421  
138 to 399 percent of the poverty threshold               ±139,284  
At or above 400 percent of the poverty threshold          ±585,728  
Below 100 percent of the poverty threshold                ±217,588  

[72 rows x 4 columns]

#### updated insurance rates:
For this we use data reported in https://www.cbpp.org/research/health/medicaid-work-requirements-could-put-36-million-people-at-risk-of-losing-health

In [6]:
# death rate data from CDC National Center for Health Statistics (https://www.cdc.gov/nchs/products/databriefs/db492.htm)
dr_ = pd.read_csv('data/Mortality Data 2021-2022.csv')
dr_

,Age_Group_Years,2021_Number,2021_Rate_per_100000,2022_Number,2022_Rate_per_100000
0,1-4,3816,25.0,4156,28.0
1,5-14,5975,14.3,6239,15.3
2,15-24,38307,88.9,35232,79.5
3,25-34,82274,180.8,74369,163.4
4,35-44,124939,287.9,111605,255.4
5,45-54,216037,531.0,183284,453.3
6,55-64,478171,1117.1,417541,992.1
7,65-74,724266,2151.3,668581,1978.7
8,75-84,829653,5119.4,824903,4708.2
9,85_and_older,940780,15743.3,933291,14389.6


## Analysis

### functions for additional mortality calculations

In [7]:
def scaling_factor(pcai,puai,hr,vec=False):
  if vec:
    scaling_factor = (puai.T + (1-puai).T.mul(hr))/(pcai.T + (1-pcai).T.mul(hr))
  else:
    scaling_factor = (puai+hr*(1-puai))/(pcai+hr*(1-pcai))
  return scaling_factor

# functions (Is this working for all scenarios)
def excess_deaths(Pa,dr,pcai,puai,hr,vec=False):
    Da = dr*Pa
    if vec:
      Du = (scaling_factor(pcai,puai,hr,vec).T*Da)
    else:
      Du = scaling_factor(pcai,puai,hr,vec)*Da
    return Da, Du, Du-Da


### function to create distribution for insurance rates.

In [8]:
def normal(value,ci,alp=0.975):
  mean = np.log(value)
  # Parameters of the normal distribution
  lower_ci = np.log(ci[0])
  upper_ci = np.log(ci[1])
  # Calculate the standard deviation using the confidence interval
  std = (upper_ci - lower_ci) / 2 / stats.norm.ppf(alp)
  retval = stats.norm(loc=mean, scale=std)
  retval.mode = np.exp(mean)
  prng = RandomState(5)
  x= np.exp(prng.normal(mean, std, 1000))
  #if restrictbelow1:
  #  x[x>1] = 1
  retval.sample = x

  return retval

# ACS survey gives us 90% CI
def get_ins(p,i,pm,im):
  retval = {}
  pD = normal(p,[p-pm,p+pm],0.95)
  pI = normal(i,[i-im,i+im],0.95)
  retval['mode'] = pI.mode/pD.mode
  retval['sample'] = pI.sample/pD.sample

  return retval

### hazard ratio

In [9]:
haz_rat = normal(1.4,[1.06,1.84],0.975)
haz_rat.sample.max()

np.float64(2.327258973573018)

### impacted population

In [10]:

popX = pop_[(pop_['SEX']==0)&(pop_['ORIGIN']==0)& (pop_['RACE']==0)]
P = popX.iloc[9,23:69].sum() # 19--64 population in 2025.
P

np.int64(201448370)

### current insurance rate among impacted population

In [11]:
#of interest:
ins_.index

#age-groups of interest:
ages = ['19 to 25 years', '26 to 34 years', '35 to 44 years','45 to 54 years', '55 to 64 years'] 
ins_.loc[ages,[('United States', 'Total', 'Estimate'), ('United States', 'Insured', 'Estimate')]]

#Insurance rates by age
#Have to convert columns to numeric by removing commas first.¶
ins_.loc[ages] = ins_.loc[ages].replace({',': '', '±': ''}, regex=True)
ins_.loc[ages] = ins_.loc[ages].apply(pd.to_numeric, errors='coerce')




ins_.loc[ages]

United States                                          
                       Total                   Insured                
Label               Estimate Margin of Error  Estimate Margin of Error
19 to 25 years      29463133           25847  25345552           29354
26 to 34 years      40085780           23247  34120692           65346
35 to 44 years      42636339           10421  37175315           63842
45 to 54 years      40377526            8438  35998280           49618
55 to 64 years      42235253            4380  38819475           34733

In [12]:
ins = ins_.loc[ages].sum()
insured = ins[('United States','Insured','Estimate')]
total = ins[('United States','Total','Estimate')]
insured_margin = ins[('United States','Insured','Margin of Error')]
total_margin = ins[('United States','Total','Margin of Error')]
pci = get_ins(total,insured, total_margin, insured_margin)
pci['mode']

np.float64(0.8801901801563885)

### updated insurance rates

In [13]:
## Number of people in population impacted who may lose insurance: 
HR = 36188000
LR = 20272000

HRL = 0.1*HR # Based on last CBO result- 15 million --1.5 million --600,000 uninsured.
LRL = 0.1*LR

In [14]:
value = HRL
# so updated insurance rate in this group is
pui = {}
pui['mode'] = pci['mode']- value/P
pui['sample'] = pci['sample'] - value/P

### current death rates

In [16]:
# Death rates
dr_ip = dr_.iloc[2:7,-2:] # this can be done better
deaths = dr_ip['2022_Number'].sum()
pops = 100000*(dr_ip['2022_Number']/dr_ip['2022_Rate_per_100000']).sum()
dr = deaths/pops
dr

np.float64(0.003804846211010321)

### excess deaths calculations

In [17]:
Da, Du, Dx = excess_deaths(P,dr,pci['mode'],pui['mode'],haz_rat.mode)
Dx

np.float64(5255.716412738082)

In [18]:
Da, Du, Dx = excess_deaths(P,dr,pci['sample'],pui['sample'],haz_rat.sample)
np.quantile(Dx,[0.025,0.5,0.975])

array([ 1088.00812674,  5314.40405474, 10951.2959909 ])

In [19]:
# All scenarios 
atrisk = [20272000, 36188000]    # Shape: (1,2)
mortprop = [0.1*0.4, 0.1, 0.89*0.25, 0.25]  # Shape: (4,1)

scenarios = [a * m for a in atrisk for m in mortprop] 
# this gives me expansion-cbol,cbou,arkl,arku & total-cbol,cbou,arkl,arku

In [20]:
# create data frame for this
iterables = [["expansion","total"], ["cbo-l", "cbo-u","ark-l","ark-u"]]
index = pd.MultiIndex.from_product(iterables, names=["first", "second"])
columns = ["95% UI Lower","Median", "95% UI Upper"]
results = pd.DataFrame(index=index,columns=columns)

In [21]:
# Now start calculating it. 
for i in range(8):
    value = scenarios[i]
    pui = {}
    pui['mode'] = pci['mode']- value/P
    pui['sample'] = pci['sample'] - value/P
    Da, Du, Dx = excess_deaths(P,dr,pci['sample'],pui['sample'],haz_rat.sample)
    results.iloc[i] = np.round(np.quantile(Dx,[0.025,0.5,0.975])).astype(int) 
    #np.quantile(Dx,[0.025,0.5,0.975])
    
    

In [22]:
results['Losing insurance'] = list(map(int, scenarios))

In [23]:
results = results[['Losing insurance',"Median", "95% UI Lower", "95% UI Upper"]]


In [24]:
results

Losing insurance Median 95% UI Lower 95% UI Upper
first     second                                                   
expansion cbo-l             810880   1191          244         2454
          cbo-u            2027200   2977          609         6135
          ark-l            4510520   6624         1356        13650
          ark-u            5068000   7443         1524        15337
total     cbo-l            1447520   2126          435         4381
          cbo-u            3618800   5314         1088        10951
          ark-l            8051830  11825         2421        24367
          ark-u            9047000  13286         2720        27378

# State-level analysis

In [28]:
# read the state-level file 
df = pd.read_excel('data/data-for-state-analysis.xlsx')
df.set_index('State',inplace=True)

In [39]:
# test
sP, sp, si, spm, sim, sil, sdr = df.loc['Alabama']
pci = get_ins(sp, si,spm, sim)
value = sil
# so updated insurance rate in this group is
pui = {}
pui['mode'] = pci['mode']- value/sP
pui['sample'] = pci['sample'] - value/sP
Da, Du, Dx = excess_deaths(sP,dr,pci['sample'],pui['sample'],haz_rat.sample)
np.quantile(Dx,[0.025,0.5,0.975])

array([ 69.30455918, 336.02485417, 686.75168992])

In [42]:
# do state-level analysis
state_results = pd.DataFrame(index=df.index,columns=["95% UI Lower","Median", "95% UI Upper"])

In [46]:
for state in state_results.index:
    sP, sp, si, spm, sim, sil, sdr = df.loc[state]
    pci = get_ins(sp, si,spm, sim)
    value = sil
    # so updated insurance rate in this group is
    pui = {}
    pui['mode'] = pci['mode']- value/sP
    pui['sample'] = pci['sample'] - value/sP
    Da, Du, Dx = excess_deaths(sP,dr,pci['sample'],pui['sample'],haz_rat.sample)
    state_results.loc[state] = np.quantile(Dx,[0.025,0.5,0.975])

In [48]:
state_results.to_excel('state_results.xlsx')

In [56]:
state_results['Median'].sort_values().sum()

53466.80441142746